### Simple demonstration of the use of generic dask functions

In [ ]:
import os
import sys

import numpy

import astropy.units as u
from astropy.coordinates import SkyCoord

from arl.graphs.execute import arlexecute
from arl.data.polarisation import PolarisationFrame
from arl.graphs.generic_graphs import create_generic_image_graph
from arl.graphs.dask_init import get_dask_Client
from arl.util.testing_support import create_test_image

In [ ]:
frequency = numpy.array([1e8])
phasecentre = SkyCoord(ra=+15.0 * u.deg, dec=-35.0 * u.deg, frame='icrs', equinox='J2000')
model_graph = arlexecute.execute(create_test_image)(frequency=frequency, phasecentre=phasecentre, cellsize=0.001,
                                         polarisation_frame=PolarisationFrame('stokesI'))
model = arlexecute.get(model_graph)


Define a simple function to take the square root of an image

In [ ]:
def imagerooter(im, **kwargs):
    im.data = numpy.sqrt(numpy.abs(im.data))
    return im

root = create_generic_image_graph(imagerooter, model_graph, facets=4)

Run the graph directly

In [ ]:
result = arlexecute.get(root)
numpy.testing.assert_array_almost_equal_nulp(result.data**2, numpy.abs(model.data), 7)

Run via local cluster

In [ ]:
client = get_dask_Client()
result=client.compute(root, sync=True)
numpy.testing.assert_array_almost_equal_nulp(result.data**2, numpy.abs(model.data), 7)
client.close()